# **LAB FOUR: RELAXATION AND CONTRAST**

This lab covers Relaxation and Contrast mechanism. Optionally, depending on the time, flow imaging can be covered.

>-------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Setup Task: Run the Notebook**
> 
> 1. Edit the cell below to set the `LAB_USER_NAME` variable to your name
> 2. Click **Run->Run All Cells** in the in top menu bar of jupyterlab
> 3. Open the Table of Contents side-bar on the left edge of jupyterlab to aid in navigation
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
LAB_USER_NAME = 'REPLACE_ME'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar.

In [ ]:
import panel as pn
pn.extension(
    inline=True, # use inline js to allow it to work offline
    raw_css=['''progress {margin: 0;}''']) # raw_css setting is a workaround for panel issue 4112
import bokeh.plotting
bokeh.plotting.output_notebook(resources=bokeh.resources.INLINE) # use inline js to allow it to work offline
import sys
import os
import yaml
import numpy as np
import matplotlib.pyplot as plt

from matipo import GLOBALS_DIR, DATA_DIR

# add inline dashboard libraries to path so they can be imported later
sys.path.append('../../../dashboards-inline')

LAB_DIR = os.path.join(DATA_DIR, LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('User data directory:', LAB_DIR)

GAMMA_BAR = 42.58e6

try:
    with open(GLOBALS_DIR+'gradient_calibration.yaml', 'r') as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)
        G_CAL = 1/(GAMMA_BAR*data['gradient_calibration']) # convert
except IOError:
    print('Unable to load gradient calibration, using default value')
    G_CAL = 0.2

try:
    with open(GLOBALS_DIR+'hardpulse_90.yaml', 'r') as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)
        AREA_90 = data['a_90']*data['t_90']
except IOError:
    print('Unable to load pulse calibration, using default value')
    AREA_90 = 0.3*32e-6
    
# Load calibrated frequency
FREQ_FILE = os.path.join(GLOBALS_DIR, 'frequency.yaml')

with open(FREQ_FILE, 'r') as f:
    freq_dict = yaml.safe_load(f)
    frequency = freq_dict['f']
    print(f"System frequency read as: {frequency*1e-6} MHz.")
    
# Load calibrated 90 and 180 hard pulses.
HP90_FILE = os.path.join(GLOBALS_DIR, 'hardpulse_90.yaml')
HP180_FILE = os.path.join(GLOBALS_DIR, 'hardpulse_180.yaml')


with open(HP90_FILE, 'r') as f:
    p_dict = yaml.safe_load(f)
    t_90 = p_dict['t_90']
    a_90 = p_dict['a_90']

with open(HP180_FILE, 'r') as f:
    p_dict = yaml.safe_load(f)
    t_180 = p_dict['t_180']
    a_180 = p_dict['a_180']

## 1. Background
Since the mechanisms of relaxation occur on the quantum level, it is important to recap how individual spins contribute to the net magnetization vector. 

Taking an averaged sum of all the quantum spins' magnetic moments ( $\vec\mu$) results in a vector that describes the net direction and magnitude of these spins - this is called the Net Magnetization Vector ($\vec M$). When placed in the magnetic field ($\vec B_0$), not all the spins will align with the field, but at thermal equilibrium their net value ($\vec M$) will align. This causes a precession around $\vec B_0$ as described in Lab One. When we add energy to the system, using the oscillating magnetic field $\vec B_1$, the tip angle of $\vec M$ can be changed. 

**Relaxation** is the process of $\vec M$ returning back to its thermal equilibrium state in alignment with $\vec B_0$.

In order to better express the different types of relaxation, $\vec M$ is split into two components: the longitudinal component ($\vec M_z$) and the transverse component ($\vec M_{xy}$). Each of these components undergo relaxation at different rates and through different mechanisms.

## 2. T1 Relaxation

This section is taken from: https://mriquestions.com/what-is-t1.html

<center><img src="images/longitudinalt1.gif" width="600"></center>
<center><figcaption style="width: 600px;">Figure 1: Longitudinal component of the magnetization recovering after excitation.</figcaption></center>

T1 relaxation is the process by which the net magnetization (M) grows/returns to its initial maximum value (Mo) parallel to Bo. Synonyms for T1 relaxation include longitudinal relaxation, thermal relaxation and spin-lattice relaxation.

$$M_z(t) = M_0 \cdot (1-e^{-t/T_1})$$

<center><img src="images/t1_relaxcurve.gif" width="400"></center>
<center><figcaption style="width: 600px;">Figure 2: T1 recovery curve.</figcaption></center>

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2.1: FID and the concept of repetition time (TR)**
> 1. Insert the Shim Sample at the correct depth.
> 2. Set number of repetitions to 5. Set the repetition time to 1 seconds. Hit scan.
> 3. Record the amplitude of first peak and the second peak. Save the plot.
> 4. Now, change TR to 0.5 secs. Repeat step 3. What do you observe?
> 5. Repeat 3 for TR=0.3, TR=0.2, TR=0.05. **Question:** Explain what you observe. Can you estimate the T1 of the sample from here? *Hint: Think about tracking the magnetization as you keep applying 90 degree pulses.* 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
TR1 = pn.widgets.FloatInput(name='TR [s]', start=0.05, end=1, step=0.1, value=0.5)
nrep = pn.widgets.IntInput(name='Repetition Number', start=1, end=10, step=1, value=1)

import importlib
import FID_w_repApp # from dashboards-inline directory that was added to sys.path
importlib.reload(FID_w_repApp)

t_acqdelay = 50e-6
t_dw = 10e-6
t_acq = lambda: TR1.value - t_acqdelay - 32e-6

print(f"n_smap: {np.floor((TR1.value - t_acqdelay - 32e-6)/t_dw)}")

s1_fid = FID_w_repApp.FIDApp(
    override_pars=dict(
        #f=frequency, # use value of frequency from the user input
        #a_90=a_90, # use rf amplitude from the user input
        #t_90=t_90,
        n_reps=nrep,
        n_scans=1,
        n_samples= lambda: np.floor((TR1.value - t_acqdelay - 32e-6)/t_dw), # Be sampling whole TR
        t_dw=t_dw,
        t_acqdelay=t_acqdelay,
        t_end=0   # Additional delay to make TR correct

    ),
    show_magnitude=True,
    show_complex=False
)

pn.Column(
    pn.Row(TR1, nrep),
    s1_fid.main(),
    sizing_mode='stretch_width'
)



**Inversion Recovery Sequence**

Images and the text is taken from https://mriquestions.com/what-is-ir.html.

Inversion recovery (IR) is a conventional spin echo (SE) sequence preceded by a 180° inverting pulse. In other words, if a SE sequence is denoted by **{90°−180°−echo}**, the IR sequence can be written as **180° — {90°−180°−echo}**
The time between the 180° inverting pulse and the 90°-pulse is called the **inversion time (TI)**. The repetition time (TR) and echo time (TE) are defined as they are for spin echo. 

<center><img src="images/ir_pulsediag.gif" width="600"></center>
<center><figcaption style="width: 600px;">Figure 3: Inversion recovery pulse diagram compared to a conventional spin echo. Notice the initial 180° inverting pulse.</figcaption></center>

The function of the inverting pulse is to flip the initial longitudinal magnetization (Mo) of all tissues in the imaged slice or volume to point opposite to the direction of the main magnetic field (Bo). During the TI interval, these inverted tissues undergo T1 relaxation as they variably seek to re-establish magnetization along the +z-direction. When spin echo signal generation begins (at the 90°-pulse), the initial longitudinal magnetizations of different tissues are now separated based on their different intrinsic T1 relaxation times. The degree of separation (and hence image contrast) is controlled by varying the TI parameter in the pulse sequence. Additional contrast effects are also obtained by manipulation of TR and TE.  

<center><img src="images/ir_recov.gif" width="600"></center>
<center><figcaption style="width: 600px;">Figure 4: Recovery of the Mz after inversion. Notice how tissues with different T1 recover with different rates. This can be exploited to create contrast and separate tissues by their T1. Also notice the point where Mz crosses 0.</figcaption></center>


> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2.2: Inversion recovery**
> 1. Put the shim phantom in.
> 2. Run the scan while sweeping through 5-6 inversion times between 0.05 and 0.5. Record the signal magnitude for each echo (magnitude at the peak).
> 3. **Question:** How does signal amplitude behaves as you increase the TI. Is it always increasing?
> 4. Find the inversion time where there is almost no signal, write this number down. **Question:** Can you think of a way how this point can be used?
> 5. Plot the numbers you acquired at step 2 versus the inversion times. **Question:** Does the plotted curve make sense? How can you measure the T1 using this curve? What is the approximate T1 by looking at the curve?
> 6. Repeat 4 and 5 for the other two phantoms with different T1's, and observe their relaxation curves and null points.
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
TI = pn.widgets.FloatInput(name='Inversion Time [s]', start=0.05, end=1, step=0.1, value=0.1)
from IRFIDApp import IRFIDApp
s2_fid = IRFIDApp(
    override_pars=dict(
        #f=frequency, # use value of frequency from the user input
        #a_90=a_90, # use rf amplitude from the user input
        #t_90=t_90,
        a_180=a_180,
        t_180=t_180,
        n_reps=1,
        n_scans=1,
        n_echo=1,
        n_samples=1000,
        t_inv=lambda: [TI.value],
        t_dw=5e-6,
        t_acqdelay=50e-6,
        t_echo=0.5e-3,
        t_end=1
    ),
    show_magnitude=True,
    show_complex=True
)

print(s2_fid.seq.par)

pn.Column(
    pn.Row(TI),
    s2_fid.main(),
    sizing_mode='stretch_width'
)

## 3. T2 Relaxation
This section is taken from: https://mriquestions.com/what-is-t2.html and https://mriquestions.com/se-vs-multi-se-vs-fse.html

T2 relaxation is the process by which the transverse components of magnetization (Mxy) decay or dephase.  As originally described by Felix Bloch (1946), T2 relaxation is considered to follow first order kinetics, resulting in a simple exponential decay (like a radio-isotope) with time constant T2. Thus T2 is the time required for the transverse magnetization to fall to approximately 37% (1/e) of its initial value. Synonyms for T2 relaxation are transverse relaxation and spin-spin relaxation.

<center><img src="images/t2_dephase.gif" width="600"></center>
<center><figcaption style="width: 600px;">Figure 5: Dephasing due to T2 relaxation.</figcaption></center>

$$M_{xy}(t) = M_{xy}(0) \cdot e^{-t/T_2}$$

### Spin Echo

The simplest form of the spin-echo (SE) pulse sequence consists of 90°-pulse, a 180°-pulse, and then an echo. The time between the middle of the first RF pulse and the peak of the spin echo is called the echo time (TE).  The sequence then repeats at time TR, the repetition time.

<center><img src="images/se_diag.gif" width="600"></center>
<center><figcaption style="width: 600px;">Figure 6: Spin-echo pulse diagram.</figcaption></center>

As described in a previous Q&A, the 180°-pulse allows refocusing of nonmoving spins whose phases have been scattered by constant field distortions and inhomogeneities. The pulse does not correct for T1 or true T2 effects due to random processes at the atomic/molecular level. It does not correct for phase shifts of spins that move, flow, diffuse or undergo chemical exchange. But if the spin has dephased simply because it is sitting near a chunk of iron-containing hemosiderin or near some imperfection in the main magnetic field of any cause, the 180°-pulse will correct for that!

<center><img src="images/se_spins.gif" width="400"></center>
<center><figcaption style="width: 600px;">Figure 7: Forming of the spin-echo.</figcaption></center>

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.1: Spin Echo Experiment**
> 1. Insert the shim sample. Run the sequence.
> 2. Observe the echo. **Question:** What governs the decay around the echo?
> 3. Sweep the echo time in the range 10-250 ms, acquire ~10 echo times. Write down the echo times and the signal amplitude at the echo.
> 4. **Question:** Plot the signal amplitudes vs. echo time. What is the number where the signal approximately drops to 37% of the first echo's amplitude. Does it match with what it says on the label of the phantom?
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
TE = pn.widgets.FloatInput(name='TE [ms]', start=10, end=1000, step=10, value=20)
print(f"TE: {TE.value*1e-3}")
from SE import SEApp
task31_se = SEApp (
            override_pars=dict(
            #f=frequency, # use value of frequency from the user input
            #a_90=a_90, # use rf amplitude from the user input
            #t_90=t_90,
            a_180=a_180,
            t_180=t_180,
            n_scans=1,
            n_samples=256,
            t_dw=5e-6,
            t_acqdelay=50e-6,
            t_echo=lambda: TE.value*1e-3,
            t_end=0.1
        ),
        show_magnitude=True,
        show_complex=False
    )

pn.Column(
    pn.Row(TE),
    task31_se.main(),
    sizing_mode='stretch_width'
)

## Multi-echo Spin-echo
As long as T2-relaxation has not completely destroyed the MR signal, it is possible to stimulate the system with additional 180°-pulse(s) and generate additional echo(es). The amplitude of each echo is progressively smaller due to T2 decay.

<center><img src="images/mcse.gif" width="600"></center>
<center><figcaption style="width: 600px;">Figure 8: Multi-echo spin-echo pulse diagram.</figcaption></center>

**CPMG** is a special type of multi-echo spin-echo, where the phase of the 180 RF pulses are altered between echoes.

By adjusting the phase of transmission, the 180° pulse may be applied along the x-axis, y-axis, or any other direction. On the left side of the diagram is a 180y° pulse, that is, one that rotates the spins around the y-axis. The echo forms in the +y-direction. On the right is a 180x° pulse that causes the echo to form along the −y-direction.

This technique reduces the accumulated phase errors and inhomogeneity effects, giving us a better estimate of the T2 decay.

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.2: Multi-echo Spin-echo (CPMG)**
> 1. Consider the T2 you found in the Task 3.1. Set the number of echoes and echo time step so that it covers your expected T2. Play with two variables until you get a good T2 decay curve.
> 2. **Question:** Does the curve look like the previous T2 curve? Considering the time it takes to use both methods, which one is better?
> When is the first method a better approach then the CPMG method? *Hint: Think about system imperfections.*
> 3. Repeat 1 for the other 2 samples with different T2 values. Record their measured T2 values.
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
TE2 = pn.widgets.FloatInput(name='TE step [ms]', start=0.5, end=50, step=1, value=1)
neco = pn.widgets.IntInput(name='Number of Echoes', start=1, end=1000, step=10, value=10)

from CPMG import CPMGApp
task32_cpmg = CPMGApp (
            override_pars=dict(
            #f=frequency, # use value of frequency from the user input
            #a_90=a_90, # use rf amplitude from the user input
            #t_90=t_90,
            a_180=a_180,
            t_180=t_180,
            n_scans=1,
            n_echo=neco,
            n_samples=64,
            t_dw=5e-6,
            t_acqdelay=50e-6,
            t_echo=lambda: TE2.value*1e-3,
            t_end=0.1
        ),
        show_magnitude=True,
        show_complex=True
    )

pn.Column(
    pn.Row(TE2, neco),
    task32_cpmg.main(),
    sizing_mode='stretch_width'
)

## 4. Flow Imaging

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 4.1:**
> 1. Go to the /notebooks/example/flow-2D-profile.ipynb and follow the instructions there.
> -------------------------------------------------------------------------------------------------------------------------------------------------------